# Open nested combat: TOP and StrokeMRI (as one dataset) open nested-harmonized to SABRE dataset

Note this must be run in the `opnc` environment

## import libraries

In [1]:
import os
import sys
from itertools import permutations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets

import neuroCombat as nC
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture
from scipy.stats import ranksums, ttest_ind, ttest_rel, ks_2samp, anderson_ksamp

sys.path.insert(0, '../') # path to cvasl functions
import cvasl.seperated as sep

## Read in our data

In [2]:
filepath = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath,'StrokeMRI_pvc2c.csv') 
filename_top = os.path.join(filepath,'TOP_pvc2c.csv') 
filename_sabre = os.path.join(filepath,'SABRE_pvc2_cleaned.csv')

In [3]:
mri = pd.read_csv(filename_mri)
top = pd.read_csv(filename_top)
sabre = pd.read_csv(filename_sabre)

In [4]:
columns = mri.columns
sabre = sabre[columns]
sabre.head(3)

,Unnamed: 0,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,...,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
0,0,sub-100096_1_ses-1_run-1,78,M,0.56201,0.44945,0.42419,0.39147,0.39147,4.621,...,13.6710,3.5865,3.8956,4.3534,3.6690,15.3022,71.6047,62.0948,43.2926,54.7345
1,1,sub-100331_1_ses-1_run-1,71,M,0.53812,0.42593,0.49319,0.36927,0.36927,1.085,...,16.3956,4.3543,4.1134,3.8613,3.7593,11.3323,37.0396,30.0755,25.5827,29.7665
2,2,sub-102285_1_ses-1_run-1,72,M,0.56881,0.49224,0.35808,0.40082,0.40082,1.956,...,20.5471,4.3167,4.4410,3.9533,3.9991,14.2770,51.7767,41.2339,33.5018,39.2296


In [5]:
mri.tail(5)

,Unnamed: 0,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,...,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
509,583,sub-59440_1_ses-1_run-1,73.928767,M,0.57259,0.52662,0.51982,0.35366,0.35366,26.949,...,2.2578,2.0305,2.0193,1.9228,2.0773,27.9190,82.7343,72.6401,54.3780,59.3805
510,584,sub-59440_2_ses-2_run-1,74.769863,M,0.57528,0.50907,0.52840,0.35671,0.35671,22.915,...,2.3344,2.5824,2.6416,2.2723,2.6217,28.7051,82.7827,76.7499,65.4477,61.8818
511,586,sub-59441_2_ses-2_run-1,74.512329,M,0.58675,0.47585,0.42899,0.39337,0.39337,5.249,...,3.8390,1.8317,1.8702,1.6799,1.7434,27.1356,66.0938,63.8049,52.7282,54.3541
512,587,sub-59442_1_ses-1_run-1,67.526027,M,0.63929,0.55235,0.39661,0.40251,0.40251,8.612,...,4.9303,1.6693,1.9822,2.1802,1.8634,24.0485,71.8750,65.1286,56.4845,59.2195
513,588,sub-59442_2_ses-2_run-1,68.410959,M,0.62909,0.53857,0.43070,0.39358,0.39358,8.944,...,4.2372,1.6986,2.1143,2.1375,1.9200,26.6148,82.6297,70.4016,58.7551,64.2534


In [6]:
topmri= pd.concat([mri,top])
topmri.tail(5)

,Unnamed: 0,participant_id,age,sex,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,...,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
522,526,sub-1163_1_ses-1_run-1,19.06,F,0.78201,0.49489,0.24608,0.51347,0.51347,1.431,...,8.2103,1.9598,2.1176,2.2812,2.1436,29.0297,117.4774,107.4787,84.1319,96.4466
523,527,sub-1165_1_ses-1_run-1,33.86,F,0.71537,0.48601,0.27123,0.48578,0.48578,1.425,...,6.6858,1.4851,1.9477,1.4834,1.8447,25.7284,87.0537,85.7033,66.4337,77.1030
524,528,sub-1167_1_ses-1_run-1,33.92,M,0.73473,0.53549,0.35985,0.45074,0.45074,3.935,...,4.5665,1.8432,2.0176,2.0263,1.9480,21.4760,68.3383,63.4410,43.3879,54.4204
525,529,sub-1168_1_ses-1_run-1,45.31,M,0.71094,0.60974,0.37480,0.41931,0.41931,4.170,...,9.5267,1.7798,1.9444,1.7670,1.8076,21.0647,61.4625,56.6210,43.5347,51.4417
526,530,sub-1169_1_ses-1_run-1,34.05,F,0.67941,0.50902,0.30031,0.45637,0.45637,2.023,...,7.0039,1.9012,2.1499,2.0002,2.2066,20.6251,74.1970,70.7366,46.0536,57.9078


## Below are functions out of the Hannah Horng Opn-combat library
The library is here https://github.com/hannah-horng/opnested-combat
As the library is unreleased and unversioned, we are using the MIT lisenced functions directly to version control them

In [7]:
# functions from opncombat
def OPNestedComBat(dat, covars, batch_list, filepath, categorical_cols=None, continuous_cols=None, return_estimates=False):
    """
    Completes sequential OPNested ComBat harmonization on an input DataFrame. Order is determined by running through all
    possible permutations of the order, then picking the order with the lowest number of features with significant
    differences in distribution.

    Arguments
    ---------
    dat : DataFrame of original data with shape (features, samples)
    covars : DataFrame with shape (samples, covariates) corresponding to original data. All variables should be label-
        encoded (i.e. strings converted to integer designations)
    batch_list : list of strings indicating batch effect column names within covars (i.e. ['Manufacturer', 'CE'...])
    filepath : root directory path for saving KS test p-values and kernel density plots created during harmonization
    categorical_cols : string or list of strings of categorical variables to adjust for
    continuous_cols : string or list of strings of continuous variables to adjust for
    return_estimates : if True, function will return both output_df and final_estimates

    Returns
    -------
    output_df : DataFrame with shape (features, samples) that has been sequentially harmonized with Nested ComBat
    final_estimates : list of dictionaries of estimates from iterative harmonization, used if user is deriving estimates
        from training data that need to be applied to a separate validation dataset

    """
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    perm_list = list(permutations(np.arange(len(batch_list))))
    count_dict = {}
    feature_dict = {}
    estimate_dict = {}
    c = 0
    for order in perm_list:
        c += 1
        n_dat = dat.copy()
        estimate_list = []
        print('Iteration ' + str(c) + ' of ' + str(len(perm_list)))
        for i in order:
            batch_col = batch_list[i]
            output = nC.neuroCombat(n_dat, covars, batch_col, continuous_cols=continuous_cols,
                                    categorical_cols=categorical_cols)
            output_df = pd.DataFrame.from_records(output['data'].T)
            n_dat = output_df.T
            estimate_list.append(output['estimates'])
        output_df.columns = dat.index
        feature_dict[str(order)] = n_dat
        count_dict[str(order)] = 0
        estimate_dict[str(order)] = estimate_list
        for batch_col in batch_list:
            p_list = []
            # print(batch_col)
            for j in range(len(output_df.columns)):
                feature = output_df.iloc[:, j]
                # print(j)
                split_col = [feature[covars[batch_col] == i] for i in covars[batch_col].unique()]
                p_list.append(anderson_ksamp(split_col).significance_level)
            count_dict[str(order)] += np.sum(np.asarray(p_list) < 0.05)
    if len(batch_list) != 1:
        best_order = [key for key, value in count_dict.items() if value == min(count_dict.values())][0]
        best_order_list = list(map(int, best_order[1:-1].split(', ')))
        order = [batch_list[i] for i in best_order_list]
        n_dat = feature_dict[best_order]
        final_estimate = estimate_dict[best_order] 

    print('Final Order: ' + str(order))

    txt_path = filepath + 'order.txt'
    with open(txt_path, 'w') as f:
        for item in order:
            f.write("%s\n" % item)

    output_df = pd.DataFrame.from_records(n_dat.T)
    output_df.columns = dat.index
    if return_estimates:
        return output_df, final_estimate
    else:
        return output_df


def feature_ad(dat, output_df, covars, batch_list, filepath):
    """
    Computes AD test p-values separated by batch effect groups for a dataset (intended to assess differences in
    distribution to all batch effects in batch_list following harmonization NestedComBat

    Arguments
    ---------
    dat : DataFrame of original data with shape (samples, features)
    output_df: DataFrame of harmonized data with shape (samples, features)
    covars : DataFrame with shape (samples, covariates) corresponding to original data. All variables should be label-
            encoded (i.e. strings converted to integer designations)
    batch_list : list of strings indicating batch effect column names within covars (i.e. ['Manufacturer', 'CE'...])
    filepath : write destination for kernel density plots and p-values

    If a feature is all the same value, the AD test cannot be completed.

    """
    p_df_original = pd.DataFrame()
    p_df_combat = pd.DataFrame()
    for batch_col in batch_list:

        # Computing KS Test P-Values
        p_list_original = []
        p_list_combat = []
        for j in range(len(output_df.columns)):
            feature_original = dat.iloc[:, j]
            feature_combat = output_df.iloc[:, j]
            try:
                split_col_original = [feature_original[covars[batch_col] == i] for i in covars[batch_col].unique()]
                p_list_original.append(anderson_ksamp(split_col_original).significance_level)
                split_col_combat = [feature_combat[covars[batch_col] == i] for i in covars[batch_col].unique()]
                p_list_combat.append(anderson_ksamp(split_col_combat).significance_level)
            except ValueError:
                print('Feature is all same value: ' + output_df.columns[j])

        p_df_original[batch_col] = p_list_original
        p_df_combat[batch_col] = p_list_combat

    p_df_original.index = dat.columns
    p_df_combat.index = output_df.columns
    p_df_original.to_csv(filepath + 'p_values_original.csv')
    p_df_combat.to_csv(filepath + 'p_values_combat.csv')
    

def feature_histograms(dat, output_df, covars, batch_list, filepath):
    """
    Plots kernel density plots and computes KS test p-values separated by batch effect groups for a dataset (intended
    to assess differences in distribution to all batch effects in batch_list following harmonization with
    NestedComBat

    Arguments
    ---------
    dat : DataFrame of original data with shape (samples, features)
    output_df: DataFrame of harmonized data with shape (samples, features)
    covars : DataFrame with shape (samples, covariates) corresponding to original data. All variables should be label-
        encoded (i.e. strings converted to integer designations)
    batch_list : list of strings indicating batch effect column names within covars (i.e. ['Manufacturer', 'CE'...])
    filepath : write destination for kernel density plots and p-values
    ad : KS test for binary batch effects if False, Anderson-Darling test for categorical batch effects if True

    """
    for batch_col in batch_list:
        filepath3 = filepath + batch_col + '/'
        if not os.path.exists(filepath3):
            os.makedirs(filepath3)

        # Plotting Kernel Density Plots
        for col in output_df:
            feature_original = dat[col]
            feature_combat = output_df[col]

            try:

                plt.figure()
                for i in covars[batch_col].unique():
                    feature_original[covars[batch_col] == i].plot.kde(color='r', alpha=0.75)
                for i in covars[batch_col].unique():
                    feature_combat[covars[batch_col] == i].plot.kde(color='b', alpha=0.5)
                    if max(feature_combat[covars[batch_col] == i]) > 1:
                        plt.xlim([0, 200])
                plt.xlabel(col)
                plt.ylabel('Density')
                leg = ['Original', 'ComBat']
                plt.legend(leg)
                ax = plt.gca()
                leg = ax.get_legend()
                leg.legendHandles[0].set_color('r')
                leg.legendHandles[1].set_color('b')

                filename = filepath3 + 'histogram_' + col + ".png"
                plt.savefig(filename, bbox_inches='tight')
                plt.close()
                plt.close('all')
            except np.linalg.linalg.LinAlgError:
                print('Failed to plot: ' + col)


def GMMSplit(dat, caseno, filepath):
    """
    Completes Gaussian Mixture model fitting and ComBat harmonization by the resulting sample grouping. The assumption
    here is that there is an unknown batch effect causing bimodality such that we can estimate the sample groupings for
    this hidden batch effect from the distribution. This function will take in a dataset, determine the best 2-component
    Gaussian mixture model, and use the resulting sample grouping to harmonize the data with ComBat.

    Arguments
    ---------
    dat : DataFrame of original data with shape (features, samples)
    caseno : DataFrame/Series containing sample IDs (should be aligned with dat and covars), used to return sample
        grouping assignments.
    filepath : root directory path for saving the grouping and corresponding kernel density plots
    -------
    new_dat : DataFrame with shape (features, samples) that has been sequentially harmonized with Nested ComBat

    """
    # GENERATING GMM GROUPING
    data_keys = list(dat.T.keys())
    aic_values = []
    predictions = []
    col_list = []
    final_keys = []
    filepath2 = filepath+'GMM_Split/'
    if not os.path.exists(filepath2):
        os.makedirs(filepath2)

    for i in range(len(data_keys)):
        # print(col)
        feature = dat.T.iloc[:, i]
        X = pd.DataFrame({0: feature, 1: feature})
        gmix = GaussianMixture(n_components=2)
        col = data_keys[i]
        try:
            gmix.fit(X)
            results = gmix.predict(X)
            cluster_0 = X[results == 0].iloc[:, 0]
            cluster_1 = X[results == 1].iloc[:, 0]
            # print(len(cluster_0))
            if len(cluster_0) <= .25*len(caseno) or len(cluster_1) <= .25*len(caseno):
                print('Clusters unbalanced: ' + data_keys[i])
            else:
                try:
                    plt.figure()
                    cluster_0.plot.kde()
                    cluster_1.plot.kde()
                    X.iloc[:, 0].plot.kde()
                    plt.legend(['Cluster 0', 'Cluster 1', 'Original'])
                    plt.xlabel(data_keys[i])
                    filename = filepath2 + 'histogram_' + data_keys[i] + ".png"
                    plt.savefig(filename, bbox_inches='tight')
                    plt.close()
                except:
                    plt.close()
                    print('Failed to plot: ' + col)
                final_keys.append(col)
                predictions.append(results)
                aic_values.append(gmix.aic(X))
                col_list.append(col)
        except ValueError:
            print('Failed to fit: ' + col)
            # aic_values.append(np.nan)

    # Returning AIC values
    gaussian_df = pd.DataFrame({'Feature': final_keys, 'AIC': aic_values})
    best_fit = gaussian_df[gaussian_df['AIC'] == min(gaussian_df['AIC'])]['Feature'].iloc[0].strip(' ')
    best_fit_n = gaussian_df[gaussian_df['AIC'] == min(gaussian_df['AIC'])]['Feature'].index[0]
    gaussian_df.to_csv(filepath2 + 'GaussianMixture_aic_values.csv')

    # Returning patient split
    predictions_df = pd.DataFrame()
    predictions_df['Patient'] = caseno
    predictions_df['Grouping'] = predictions[best_fit_n]
    predictions_df.to_csv(filepath2 + best_fit + '_split.csv')

    return predictions_df

In [8]:
topmri_batch_testing_df = topmri[['participant_id','age', 'sex']]
topmri_batch_testing_df['site'] = 1
column_to_move = topmri_batch_testing_df.pop("site")
topmri_batch_testing_df.insert(1, "site", column_to_move)
#topmri_batch_testing_df.head(3)

C:\Users\makeda\AppData\Local\Temp\ipykernel_28368\2103901941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topmri_batch_testing_df['site'] = 1


In [9]:
sabre_batch_testing_df = sabre[['participant_id','age', 'sex']]
sabre_batch_testing_df['site'] = 2
column_to_move = sabre_batch_testing_df.pop("site")
sabre_batch_testing_df.insert(1, "site", column_to_move)
#sabre_batch_testing_df.head(3)

C:\Users\makeda\AppData\Local\Temp\ipykernel_28368\3750818050.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sabre_batch_testing_df['site'] = 2


In [10]:
batch_testing_df = pd.concat([topmri_batch_testing_df, sabre_batch_testing_df], ignore_index=True)
batch_testing_df = sep.recode_sex_to_numeric(batch_testing_df)
batch_testing_df.tail(5)

,participant_id,site,age,sex
1738,sub-96612_1_ses-1_run-1,2,72.0,0
1739,sub-96645_1_ses-1_run-1,2,73.0,0
1740,sub-97154_1_ses-1_run-1,2,71.0,0
1741,sub-97819_1_ses-1_run-1,2,72.0,0
1742,sub-97905_1_ses-1_run-1,2,71.0,0


In [11]:
# Loading in batch effects
batch_testing_list = ['site']
# Loading in clinical covariates
categorical_testing_cols = ['sex']
continuous_testing_cols = ['age']

In [12]:
# ASL
data_testing_df = pd.concat([topmri,sabre])
#data_testing_df = # ASL
data_testing_df = data_testing_df.drop(columns=['Unnamed: 0','age','sex'])  
data_testing_df.tail(3)

,participant_id,gm_vol,wm_vol,csf_vol,gm_ivc_ratio,gmwm_ivc_ratio,wmh_vol,wmh_count,deepwm_b_cov,aca_b_cov,mca_b_cov,pca_b_cov,totalgm_b_cov,deepwm_b,aca_b,mca_b,pca_b,totalgm_b
699,sub-97154_1_ses-1_run-1,0.49420,0.43000,0.26314,0.41622,0.41622,2.168,23.0,28.5449,4.0986,3.8984,2.7740,3.4137,13.4864,53.8449,46.8761,39.5245,45.0268
700,sub-97819_1_ses-1_run-1,0.58200,0.51335,0.39980,0.38926,0.38926,14.889,24.0,7.9896,3.4410,3.4441,3.6305,3.1638,18.6820,68.1887,55.7775,41.6975,50.4850
701,sub-97905_1_ses-1_run-1,0.64023,0.57521,0.48202,0.37717,0.37717,3.042,27.0,21.2033,4.5504,4.9379,4.3810,4.2747,10.5935,42.1268,36.5742,35.7132,35.3186


In [ ]:
#data_testing_df# = sep.recode_sex_to_numeric(data_testing_df)

In [ ]:
#caseno_testing 

In [ ]:
data_testing_df = data_testing_df.reset_index(drop=True)
data_testing_df = data_testing_df.dropna()
data_testing_df = data_testing_df.merge(batch_testing_df['participant_id'], 
                                        left_on='participant_id', right_on='participant_id')
dat_testing = data_testing_df.iloc[:, 1:]
dat_testing = dat_testing.T.apply(pd.to_numeric)
caseno_testing = data_testing_df['participant_id']
covars_testing = batch_testing_df.drop('participant_id',1)

In [ ]:
#dat_testing

In [ ]:
# Merging batch effects, clinical covariates
covars_testing_string = pd.DataFrame()
covars_testing_string[categorical_testing_cols] = covars_testing[categorical_testing_cols].copy()
covars_testing_quant = covars_testing[continuous_testing_cols]
#covars_testing_quant

In [ ]:
# Encoding categorical variables
covars_testing_cat = pd.DataFrame()
for col_testing in covars_testing_string:
    stringcol_testing = covars_testing_string[col_testing]
    le = LabelEncoder()
    le.fit(list(stringcol_testing))
    covars_testing_cat[col_testing] = le.transform(stringcol_testing)
#covars_testing_cat

In [ ]:
covars_testing_final = pd.concat([covars_testing_cat, covars_testing_quant], axis=1)
#covars_testing_final

In [ ]:
# # # FOR GMM COMBAT VARIANTS:
# # # Adding GMM Split to batch effects
filepath2 = 'Testing/OPPNComBat/ResultTesting'
if not os.path.exists(filepath2):
    os.makedirs(filepath2)
gmm_testing_df = GMMSplit(dat_testing, caseno_testing, filepath2)

In [ ]:
batch_testing_df.head(2)

In [ ]:
gmm_testing_df.head(2)

In [ ]:
gmm_testing_df_merge = batch_testing_df.merge(gmm_testing_df, right_on='Patient', left_on='participant_id')
gmm_testing_df_merge['GMM'] = gmm_testing_df_merge['Grouping'] 
gmm_testing_df_merge

In [ ]:
covars_testing_final = gmm_testing_df_merge.drop(['participant_id','Patient','Grouping'],1)
categorical_testing_cols = categorical_testing_cols + ['GMM']

In [ ]:
output_testing_df = OPNestedComBat(dat_testing,
                                   covars_testing_final,
                                   batch_testing_list,
                                   filepath2, categorical_cols=categorical_testing_cols,
                                  continuous_cols=continuous_testing_cols)

In [ ]:
# compute the AD test p-values to measure harmonziation performance
feature_ad(dat_testing.T, output_testing_df, covars_testing_final, batch_testing_list, filepath2)
# plot kernel density plots to visualize distributions before and after harmonization
feature_histograms(dat_testing.T, output_testing_df, covars_testing_final, batch_testing_list, filepath2)

In [ ]:
write_testing_df = pd.concat([caseno_testing, output_testing_df], axis=1) 
write_testing_df.to_csv(filepath2+'/Mfeatures_testing_NestedComBat.csv') # write results fo file
dat_testing_input = dat_testing.transpose()
dat_testing_input.to_csv(filepath2+'/Mfeatures_input_testing_NestedComBat.csv')
covars_testing_final.to_csv(filepath2+'/Mcovars_input_testing_NestedComBat.csv')

In [ ]:
# write harmonized dataset 
complete_harmonised = pd.concat([write_testing_df, covars_testing_final], axis=1) 
complete_harmonised.head(3)

# split dataframe back up into parts for running
## from complete_harmonised

In [ ]:
sabre_opn_harmonized = complete_harmonised[complete_harmonised['site'] == 2]
sabre_opn_harmonized = sabre_opn_harmonized.drop(columns=['site', 'GMM',])
#sabre_opn_harmonized.head(3)

In [ ]:
topmri_opn_harmonized = complete_harmonised[complete_harmonised['site'] == 1]
topmri_opn_harmonized = topmri_opn_harmonized.drop(columns=['site', 'GMM',])
#top_opn_harmonized.head(3)

In [ ]:
topmri_opn_harmonized.to_csv('Ltopmri_opn_harmonized.csv')
sabre_opn_harmonized.to_csv('Lsabre_opn_harmonized.csv')

In [ ]:
topmri_opn_harmonized.tail(5)

In [ ]:
# visualize_harmonized(original_df_together, complete_harmonized, 'gm_vol')

In [ ]:
# visualisation prep
complete_harmonised['Harmonised'] = 'H'
complete_nonharmonised = pd.concat([caseno_testing, dat_testing_input], axis=1) 
complete_nonharmonised = pd.concat([complete_nonharmonised, covars_testing_final], axis=1) 
complete_nonharmonised['Harmonised'] = 'UH'
complete_harmonised_nonharmonised_merged = complete_harmonised.append(complete_nonharmonised)

In [ ]:
complete_harmonised_nonharmonised_merged['Harmonised_sex'] = np.nan
complete_harmonised_nonharmonised_merged['Harmonised_sex'] = np.where((complete_harmonised_nonharmonised_merged['sex'] == 0) & (complete_harmonised_nonharmonised_merged['Harmonised'] == 'H'), 'H_Male', complete_harmonised_nonharmonised_merged['Harmonised_sex'])
complete_harmonised_nonharmonised_merged['Harmonised_sex'] = np.where((complete_harmonised_nonharmonised_merged['sex'] == 1) & (complete_harmonised_nonharmonised_merged['Harmonised'] == 'H'), 'H_Female', complete_harmonised_nonharmonised_merged['Harmonised_sex'])
complete_harmonised_nonharmonised_merged['Harmonised_sex'] = np.where((complete_harmonised_nonharmonised_merged['sex'] == 0) & (complete_harmonised_nonharmonised_merged['Harmonised'] == 'UH'), 'UH_Male', complete_harmonised_nonharmonised_merged['Harmonised_sex'])
complete_harmonised_nonharmonised_merged['Harmonised_sex'] = np.where((complete_harmonised_nonharmonised_merged['sex'] == 1) & (complete_harmonised_nonharmonised_merged['Harmonised'] == 'UH'), 'UH_Female', complete_harmonised_nonharmonised_merged['Harmonised_sex'])
complete_harmonised_nonharmonised_merged['Harmonisation'] = complete_harmonised_nonharmonised_merged['Harmonised']


In [ ]:
#complete_harmonised_nonharmonised_merged.columns

In [ ]:
# Pick something to visualize effects

In [ ]:
features = widgets.Dropdown(
    options=complete_harmonised_nonharmonised_merged.columns.tolist(),
    value='gm_vol',
    description='Feature',
    disabled=False
)
features

In [ ]:
feature =features.value
y_axis = feature
viz_data = complete_harmonised_nonharmonised_merged
#y_axis = "totalgm_b_cov"
sns.set_style("whitegrid")
sns.catplot(data = complete_harmonised_nonharmonised_merged, x = 'Harmonisation', y = y_axis, hue ="sex", 
split=True, inner = 'quartile', kind = 'violin', 
col = 'site', height = 5, aspect = 0.6)
plt.ylim((viz_data[y_axis].min(),viz_data[y_axis].max()))
plt.show()

<!-- # split dataframe back up into parts for running
## from complete_harmonised -->